In [2]:
import torch
import torch.nn.functional as F
import sympy as sp
import pandas as pd

In [3]:
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")

In [4]:
df = pd.read_csv('hundo_equations.csv')

functions = []
num_vars_per_func = []

for _, row in df.iterrows():
    formula = row['Formula']
    num_vars = row['# variables']
    function_details = {
        'formula': formula,
        'variables': []
    }
    
    for i in range(1, 6):
        v_name = row.get(f'v{i}_name')
        v_low = row.get(f'v{i}_low')
        v_high = row.get(f'v{i}_high')
        
        if pd.notna(v_name):
            function_details['variables'].append({
                'name': v_name,
                'low': v_low,
                'high': v_high
            })
    
    functions.append(function_details)
    num_vars_per_func.append(num_vars)

for i, func in enumerate(functions):
    print(f"Function {i+1}:")
    print(f"  Formula: {func['formula']}")
    print(f"  Number of Variables: {num_vars_per_func[i]}")
    print("  Variables:")
    for var in func['variables']:
        print(f"    - Name: {var['name']}, Range: ({var['low']}, {var['high']})")
    print()

ParserError: Error tokenizing data. C error: Expected 20 fields in line 3, saw 23


In [13]:
def generate_function(function, sample_size, device):
    hold = []
    sympy_symbols = []
    
    for var in function["variables"]:
        sym = sp.symbols(var["name"])
        sympy_symbols.append(sym)
        min_val = var["low"]
        max_val = var["high"]
        v = (max_val - min_val) * torch.rand(sample_size, 1, 1) + min_val
        hold.append(v)
    
    params = torch.stack(hold)
    params = torch.atleast_2d(params)
    params = torch.transpose(params, 0, 1)
    params = torch.transpose(params, 1, 2)
    params = params.to(device)

    formula = sp.sympify(function["formula"])
    
    results = []
    for i in range(sample_size):
        var_values = {sympy_symbols[j]: params[i, 0, j].item() for j in range(len(sympy_symbols))}
        evaluated = formula.subs(var_values)
        results.append(float(evaluated))

    print(params)
    
    return results, formula, sympy_symbols, params

In [14]:
sample_size = 1000
hold = []
for f in functions:
    try:
        results = generate_function(f, sample_size, device)
        hold.append(results)
    except Exception as e:
        print(f"Error processing function {f}: {e}")

Error processing function {'formula': 2, 'variables': [{'name': -10.0, 'low': 10, 'high': 'b'}, {'name': -10.0, 'low': 10.0, 'high': nan}]}: 'float' object is not iterable
Error processing function {'formula': 3, 'variables': [{'name': -10.0, 'low': 10, 'high': 'b'}, {'name': -10.0, 'low': 10.0, 'high': 'c'}, {'name': -10.0, 'low': 10.0, 'high': nan}]}: 'float' object is not iterable
Error processing function {'formula': 4, 'variables': [{'name': -10.0, 'low': 10, 'high': 'b'}, {'name': -10.0, 'low': 10.0, 'high': 'c'}, {'name': -10.0, 'low': 10.0, 'high': 'd'}, {'name': -10.0, 'low': 10.0, 'high': nan}]}: 'float' object is not iterable
Error processing function {'formula': 1, 'variables': [{'name': 0.1, 'low': 10, 'high': nan}]}: 'float' object is not iterable
Error processing function {'formula': 2, 'variables': [{'name': 0.1, 'low': 10, 'high': 'b'}, {'name': 0.1, 'low': 10.0, 'high': nan}]}: 'float' object is not iterable
Error processing function {'formula': 1, 'variables': [{'nam

In [7]:
hold_data = [torch.tensor(l[0]) for l in hold]
hold_formulas = [l[1] for l in hold]
hold_symbols = [l[2] for l in hold]
hold_params = [l[3] for l in hold]
hold_num_params = [len(l[2]) for l in hold]

In [12]:
hold

[]

In [7]:
def compute_derivative(parameters, func, symbols, formula, epsilon=1e-6):
    batch_size = parameters.shape[0]
    num_params = parameters.shape[2]
    gradients = torch.zeros(batch_size, num_params)

    for i in range(batch_size):
        param_tensor = parameters[i].clone().detach().requires_grad_(True)
        
        for j in range(num_params):
            perturbed_params_pos = param_tensor.clone()
            perturbed_params_neg = param_tensor.clone()
            
            perturbed_params_pos[0, j, 0] += epsilon
            forward_value = func(perturbed_params_pos, symbols, formula)
            
            perturbed_params_neg[0, j, 0] -= epsilon
            backward_value = func(perturbed_params_neg, symbols, formula)
            
            gradients[i, j] = (forward_value - backward_value) / (2 * epsilon)
    
    return gradients


In [8]:
def compute_hessians(parameters, func, symbols, formula, epsilon=1e-6): #need to fix
    batch_size = parameters.shape[0]
    max_num_params = parameters.shape[2]
    hessians = torch.zeros((batch_size, max_num_params, max_num_params))
    
    for i in range(batch_size):
        param_tensor = parameters[i].clone().detach().requires_grad_(True)
        
        for j in range(max_num_params):
            for k in range(max_num_params):
                # Perturb j-th and k-th parameters
                perturbed_params = param_tensor.clone()
                
                # Compute f(x + epsilon * e_j + epsilon * e_k)
                perturbed_params_jk = perturbed_params.clone()
                perturbed_params_jk[:, j, :] += epsilon
                perturbed_params_jk[:, k, :] += epsilon
                f_plus_plus = func(perturbed_params_jk, symbols, formula)
                
                # Compute f(x + epsilon * e_j - epsilon * e_k)
                perturbed_params_jk[:, k, :] -= 2 * epsilon
                f_plus_minus = func(perturbed_params_jk, symbols, formula)
                
                # Compute f(x - epsilon * e_j + epsilon * e_k)
                perturbed_params_jk[:, j, :] -= 2 * epsilon
                perturbed_params_jk[:, k, :] += 2 * epsilon
                f_minus_plus = func(perturbed_params_jk, symbols, formula)
                
                # Compute f(x - epsilon * e_j - epsilon * e_k)
                perturbed_params_jk[:, k, :] -= 2 * epsilon
                f_minus_minus = func(perturbed_params_jk, symbols, formula)
                hessians[i, j, k] = (f_plus_plus - f_plus_minus - f_minus_plus + f_minus_minus) / (4 * epsilon**2)
    
    return hessians

In [9]:
def evaluate_function(params, symbols, formula):
    if params.shape[0] != 1:
        var_values = {symbols[j]: params[j].item() for j in range(len(symbols))}
    else:
        var_values = {symbols[j]: params[:, j, :].item() for j in range(len(symbols))}
    evaluated = formula.subs(var_values)
    return float(evaluated)

In [10]:
derivatives = [compute_derivative(hold_params[i], evaluate_function, hold_symbols[i], hold_formulas[i]) for i in range(len(hold_params))]
hessians = [compute_hessians(hold_params[i], evaluate_function, hold_symbols[i], hold_formulas[i]) for i in range(len(hold_params))]

In [42]:
flat_result = torch.flatten(torch.stack(hold_data).unsqueeze(2), start_dim=0, end_dim=1)

max_der = max(d.size(1) for d in derivatives)
padded_ders = []
for d in derivatives:
    padded_tensor = F.pad(d, pad=(0, max_der - d.size(1)))
    padded_ders.append(padded_tensor)
flat_der = torch.flatten(torch.stack(padded_ders), start_dim=0, end_dim=1)

max_hess = max_der**2
padded_hess = []
for h in hessians:
    h = torch.flatten(h, start_dim=1)
    padded_tensor = F.pad(h, pad=(0, max_hess - h.size(1)))
    padded_hess.append(padded_tensor)
flat_hess = torch.flatten(torch.stack(padded_hess), start_dim=0, end_dim=1)

flat_der = F.pad(flat_der, (0, flat_hess.size(-1) - flat_der.size(-1)))
flat_result = F.pad(flat_result, (0, flat_hess.size(-1) - flat_result.size(-1)))

flat_result.shape, flat_der.shape, flat_hess.shape

(torch.Size([10000, 81]), torch.Size([10000, 81]), torch.Size([10000, 81]))

In [44]:
torch.save({
    'flattened_data': torch.stack([flat_result, flat_der, flat_hess], dim=2),
    'results': hold_data,
    'formulas': hold_formulas,
    'symbols': hold_symbols,
    'params': hold_params,
    'num_params': hold_num_params,
}, 'hold_data.pth')